In [10]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as sm
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
from datetime import date
import matplotlib.pyplot as plt
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="p0o9i8u7",
  database="jockey"
)

#file0 = "/Users/apple/Workspace/intelliJ/jockey/regression_20190127.csv"

#dataset0 = pd.read_csv(file0)

#'horse_winCount','jockey_winCount',,'declaredHorseWeight','addedWeight'
#
#features=['draw','horse_winPer','jockey_winPer','horse_newDistance','horse_newHorse',
#         'horse_last4SpeedRate','horse_latestSpeedRate','Days_from_lastRace','weightRadio','weightRD',
#           'horseFx','horseFx_Distance','jockeyFx','jockeyFx_Distance','propByWinOdds']

features=['draw','horse_newDistance','horse_newHorse',
         'horse_last4SpeedRate','horse_latestSpeedRate','Days_from_lastRace','weightRD',
           'horseFx','horseFx_Distance','jockeyFx','jockeyFx_Distance','propByWinOdds']

target_fea='finishTime'
target=[target_fea]
sql = 'select raceDate,raceSeqOfDay,distance,raceClass, horseId, place,lbs,winOdds,'
for aFeature in features:
    sql = sql + aFeature +", "
sql = sql+ target_fea+ " from racecard"
#where racedate='20190213'"

columns=['raceDate','raceSeqOfDay','distance','raceClass',*features,*target]

dataset0 = pd.read_sql(sql,mydb)
#print(dataset0)
#allDates =['20190217']
allDates= dataset0.raceDate.unique()
allDates.sort()
tableName='racecard'
#dataset0
updatedate='20190322'


In [11]:

for raceDate in allDates:
    if(raceDate < '20180909'):
         continue
    print(raceDate)
    dataset_ofADay = dataset0[(dataset0['raceDate']==raceDate)]
    
    allSeqs = dataset_ofADay.raceSeqOfDay.unique()

    for seq in allSeqs:
        dataset_test =  dataset_ofADay[(dataset_ofADay['raceSeqOfDay']==seq)]
        distance = dataset_test['distance'].iloc[0]
        raceClass = dataset_test['raceClass'].iloc[0]
        print(raceDate,seq,distance,raceClass)
        
        dataset_train = dataset0[(dataset0['raceClass']==raceClass) & (dataset0['distance']==distance) 
                        & (dataset0['raceDate']!=raceDate)]
        #dataset_train.shape
        if(dataset_train.size==0):
            continue
        #regression model of lbw or finishTime
        X_train=dataset_train[features]
        y_train=dataset_train[target] #lbw

        X_test=dataset_test[features]
        y_test=dataset_test[target] #lbw


        reg = LinearRegression()
        reg.fit(X_train,y_train)
        y_test_predicted=reg.predict(X_test)
        
        #plt.scatter(y_test, y_test_predicted,  color='black')
        #plt.show()
        #print("Mean squared error: %.2f"
        #        % mean_squared_error(y_test, y_test_predicted))
        # Explained variance score: 1 is perfect prediction
        #print('Variance score: %.2f' % r2_score(y_test, y_test_predicted))
        #print('explained_variance_score score: %.2f' % explained_variance_score(y_test, y_test_predicted))
         #explained_variance_score(y_test, y_test_predicted)
        df = pd.DataFrame(data=y_test_predicted, index=y_test.index,columns=['finishTime_predicted'])
        dataset_test_toshow=dataset_test[['raceDate','raceSeqOfDay','distance','raceClass','horseId','place','lbs','finishTime']]
        dataset_test_withPredicted=dataset_test_toshow.merge(df,how='inner',left_index=True,right_index=True)
        dataset_test_withPredicted = dataset_test_withPredicted.sort_values(['finishTime_predicted'])
        #print(dataset_test_withPredicted)
       
        predicted_place=1
        for index, row in dataset_test_withPredicted.iterrows():
            mycursor = mydb.cursor()
            raceDate_str = str(row['raceDate'])
            raceSeqOfDate =str(row['raceSeqOfDay'])
            horseId = str(row['horseId'])
            finishTime_predicted =str(row['finishTime_predicted'])
            ##print("Andrea:",raceDate_str,raceSeqOfDate,horseId,finishTime_predicted)
            sql = ("UPDATE "+tableName+ " SET predicted_date=" + updatedate +", predicted_place="+ str(predicted_place) +", predicted_finishTime = "+ finishTime_predicted +
            " WHERE raceDate = "+ raceDate_str + " and raceSeqOfDay = "+raceSeqOfDate + " and horseId ='"+horseId+ "' ")
            
            #print(predicted_place)
            predicted_place =predicted_place+1
            #print(sql)
            mycursor.execute(sql)
            mydb.commit()
            #print(mycursor.rowcount, " record(s) updated" ,row['raceSeqOfDay'] ,",", row['horseId'] )


20180909
20180909 1 1000 5
20180909 2 1400 1
20180909 3 1600 3
20180909 4 1400 5
20180909 5 1200 4
20180909 6 1200 4
20180909 7 1600 4
20180909 8 1400 4
20180909 9 1200 3
20180909 10 1200 3
20180912
20180912 1 1800 5
20180912 2 1200 4
20180912 3 1650 3
20180912 4 1200 4
20180912 5 1650 4
20180912 6 1000 3
20180912 7 1200 3
20180912 8 1650 2
20180922
20180922 1 1200 4
20180922 2 1200 5
20180922 3 1200 4
20180922 4 1600 4
20180922 5 1400 4
20180922 6 1200 3
20180922 7 1800 3
20180922 8 1000 3
20180922 9 1400 3
20180922 10 1400 2
20180926
20180926 1 1000 5
20180926 2 1800 5
20180926 3 1200 4
20180926 4 1200 4
20180926 5 1650 3
20180926 6 1650 4
20180926 7 1200 3
20180926 8 1200 3
20181001
20181001 1 1000 4
20181001 2 1800 4
20181001 3 1200 4
20181001 4 1400 4
20181001 5 1400 3
20181001 6 1600 3
20181001 7 1000 3
20181001 8 1400 3
20181001 9 1200 3
20181001 10 1200 2
20181003
20181003 1 1650 5
20181003 2 1800 3
20181003 3 1650 4
20181003 4 1200 4
20181003 5 1200 4
20181003 6 1200 3
2018100

In [7]:
 #explained_variance_score(y_test, y_test_predicted)
df = pd.DataFrame(data=y_test_predicted, index=y_test.index,columns=['finishTime_predicted'])
dataset_test_toshow=dataset_test[['raceDate','raceSeqOfDay','distance','raceClass','horseId','place','winOdds','lbs','finishTime']]
dataset_test_withPredicted=dataset_test_toshow.merge(df,how='inner',left_index=True,right_index=True)
dataset_test_withPredicted = dataset_test_withPredicted.sort_values(['finishTime_predicted'])
dataset_test_withPredicted

,raceDate,raceSeqOfDay,distance,raceClass,horseId,place,winOdds,lbs,finishTime,finishTime_predicted
14773,20190217,10,1400,3,B311,7,2.0,4.00,82.87,81.811891
14772,20190217,10,1400,3,A354,6,10.0,4.00,82.85,82.522883
14767,20190217,10,1400,3,T403,1,22.0,0.00,82.22,82.536577
14769,20190217,10,1400,3,A010,3,23.0,2.25,82.58,82.536623
14775,20190217,10,1400,3,B328,9,11.0,4.25,82.91,82.537055
14776,20190217,10,1400,3,A137,10,70.0,5.00,83.00,82.630466
14780,20190217,10,1400,3,B234,14,13.0,8.25,83.53,82.686080
14778,20190217,10,1400,3,A178,12,36.0,5.00,83.03,82.709244
14774,20190217,10,1400,3,B456,8,6.9,4.25,82.89,82.788485
14768,20190217,10,1400,3,B285,2,7.5,0.50,82.30,82.798318
